# core.async

下面代码添加`clojure.core.async`到当前环境中。首次调用可能运行较长时间

In [2]:
(require ['cemerick.pomegranate :refer ['add-dependencies]])
(add-dependencies :coordinates [['org.clojure/core.async "0.2.391"]])
(require ['clojure.core.async :as 'a :refer '[>! <! >!! <!! go chan buffer close! thread alts! alts!! timeout dropping-buffer sliding-buffer]])                                

nil {[org.clojure/core.async "0.2.391"] #{[org.clojure/tools.analyzer.jvm "0.6.10"]}, [org.clojure/tools.analyzer.jvm "0.6.10"] #{[org.clojure/tools.reader "1.0.0-beta2"] [org.clojure/tools.analyzer "0.6.9"] [org.clojure/core.memoize "0.5.9"] [org.ow2.asm/asm-all "4.2"] [org.clojure/clojure "1.5.1"]}, [org.clojure/tools.analyzer "0.6.9"] nil, [org.clojure/core.memoize "0.5.9"] #{[org.clojure/core.cache "0.6.5"]}, [org.clojure/core.cache "0.6.5"] #{[org.clojure/data.priority-map "0.0.7"]}, [org.clojure/data.priority-map "0.0.7"] nil, [org.ow2.asm/asm-all "4.2"] nil, [org.clojure/tools.reader "1.0.0-beta2"] nil, [org.clojure/clojure "1.5.1"] nil}

## Chan/Go

core.async的中心是`chan`和`go`。`chan`用来收取和发送消息，`go`用来创建一个在其他thread里面跑的`进程`。

In [12]:
(def echo-chan (chan))
(go (println (<! echo-chan)))
(>!! echo-chan "hello")

hello


#'user/echo-chan #object[clojure.core.async.impl.channels.ManyToManyChannel 0x8954af7 "clojure.core.async.impl.channels.ManyToManyChannel@8954af7"] true

以上代码创建一个`chan`，使用`go`block创建一个进程，从chan中读取消息，最后给chan发送消息hello。其中

- 函数 `<!`，`<!!`用来读取消息
- `>!!` ，`>!`用来发送消息，并且它会等待消息被读取。如果使用如`(>!! (chan) "my message")`，由于没有监听在chan上的代码，这段发送消息的代码会无限阻塞

当在`go block`中时，使用一个叹号的形式，否则使用两个叹号。

chan可以有buffer，代表可缓存消息的条目，比如

In [13]:
(def buf (cha 3))
(>!! buf "1")
(>!! buf "2")

此时即使channel没有被监听，代码也不会阻塞。但是如果传入消息的次数超过的buffer的大小，代码会重新阻塞。buffer 也分为如下几种

- 普通的buffer
- sliding buffer。先进先出的丢弃消息
- dropping buffer。后进先出的丢弃消息

In [4]:
(def droping (chan (dropping-buffer 1)))
(def sliding (chan (sliding-buffer 1)))
(>!! droping "message 1")
(>!! sliding "message 1")
(>!! droping "message 2")
(>!! sliding "message 2")
(<!! droping)
(<!! sliding)

#'user/droping #'user/sliding true true true true "message 1" "message 2"

## thread

thread创建一个新的线程并运行其代码， 它的返回值是一个channel，在thread的代码结束后代码的返回值传入到channel。

In [7]:
(def echo-chan (chan))
(def t (thread (do (println (<!! echo-chan)) "cool")))
(>!! echo-chan "hell again")
(println (str "return value is " (<!! t)))

hell again
return value is cool


#'user/echo-chan #'user/t true nil